In [ ]:
# Hagamos una conversación entre GPT-4o-mini,Claude-3-haiku y Gemini 1.5 flash

# Estamos usando versiones económicas de los modelos, por lo que los costos serán mínimos
gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
gemini_model = "gemini-1.5-flash"

# Definición de personalidad para cada chatbot
gpt_system = "Eres un chatbot muy argumentativo; \
no estás de acuerdo con nada en la conversación y cuestionas todo de manera sarcástica."

claude_system = "Eres un chatbot muy educado y cortés. Intentas estar de acuerdo con \
todo lo que dice la otra persona o encontrar puntos en común. Si la otra persona discute, \
intentas calmarla y seguir charlando."

gemini_system = "Eres un chatbot humorista;\
intentas poner un toque de humor a todo lo que dicen otras personas;\
si la otra persona discute, le pones un toque de ironía y sigues charlando."

# Inicializamos las listas de mensajes de cada chatbot
gpt_messages = ["¡Hola!"]
claude_messages = ["Hola"]
gemini_messages = ["Helloooo"]

In [ ]:
# Función para hacer que GPT-4o-mini responda
def call_gpt():
        messages = [{"role": "system", "content": gpt_system}]  # Establecemos la personalidad de GPT
     # Recorremos los mensajes previos para darle contexto a GPT
    for gpt, claude, gemini in zip(gpt_messages, claude_messages,gemini_messages):
        messages.append({"role": "assistant", "content": gpt}) # Mensajes de GPT
        messages.append({"role": "user", "content":f"{claude}\n{gemini}"}) # Mensajes de Claude y Gemini juntos
    # Llamamos a la API de OpenAI para obtener la respuesta de GPT
    completion = openai.chatCompletion.create(
        model=gpt_model,
        messages=messages)
    # Devolvemos la respuesta de GPT
    return completion.["choices"][0]["message"]["content"]

In [ ]:
# Función para hacer que Claude-3-Haiku responda
def call_claude():
    messages = [] # Lista para almacenar el historial de mensajes
    # Recorremos los mensajes previos para darle contexto a Claude
    for gpt, claude, gemini in zip(gpt_messages, claude_messages,gemini_messages):
        messages.append({"role": "user", "content": f"{gpt}\n{gemini}"})  # Mensajes de GPT y Gemini juntos
        messages.append({"role": "assistant", "content": claude})  # Respuesta anterior de Claude
    messages.append({"role": "user", "content": f"{gpt_messages[-1]}\n{gemini_messages[-1]}"}) # Últimos mensajes
    # Conectamos con la API de Anthropic (Claude)
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500)
    # Devolvemos la respuesta de Claude
    return message.content[0].text

In [ ]:
# Función para hacer que Gemini 1.5 Flash responda
def call_gemini():
    messages = [{"role": "user", "content": gemini_system}] # Establecemos la personalidad de Gemini
    # Recorremos los mensajes previos para darle contexto a Gemini
    for gpt, claude, gemini in zip(gpt_messages, claude_messages,gemini_messages):
        messages.append({"role": "user", "content": f"{gpt}\n{claude}"}) # Mensajes de GPT y Claude juntos
        messages.append({"role": "assistant", "content": gemini}) # Mensajes de GPT y Claude juntos
    messages.append({"role": "user", "content": f"{gpt_messages[-1]}\n {claude_messages[-1]}"}) # Últimos mensajes
    # Conectamos con la API de Google (Gemini)
    message = genai.chat(
        model=gemini_model,
        messages = [{"role":"user","content":msg["content"]} for msg in messages])
    return.message.last # Devolvemos la última respuesta de Gemini

In [ ]:
# CICLO DE CONVERSACIÓN ENTRE LOS BOTS 
messages = [{"role": "user", "content": "¡Hola!"}]  # Mensaje inicial de la conversación

for i in range(5):  # Definimos el número de rondas de conversación
    print(f"\n Ronda {i+1}")

    # GPT responde
    gpt_response = call_gpt()
    print(f"GPT: {gpt_response}")
    messages.append({"role": "assistant", "content": gpt_response})
    gpt_messages.append(gpt_response)  # Guardamos la respuesta para contexto futuro

    # Claude responde
    claude_response = call_claude()
    print(f"Claude: {claude_response}")
    messages.append({"role": "assistant", "content": claude_response})
    claude_messages.append(claude_response)  # Guardamos la respuesta para contexto futuro

    # Gemini responde
    gemini_response = call_gemini()
    print(f"Gemini: {gemini_response}")
    messages.append({"role": "assistant", "content": gemini_response})
    gemini_messages.append(gemini_response)  # Guardamos la respuesta para contexto futuro